1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다.
    - 토큰수 초과로 답변을 생성하지 못할 수 있고
    - 문서가 길면 (input이 길면) 답변 생성이 오래 걸림
3. embedding -> vector database에 저장
4. 질문이 있을 때, vector database에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달


In [25]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('./tax.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [26]:
len(document_list)

220

In [27]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

# 환경변수를 불러옴
load_dotenv()

# OpenAI에서 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [46]:
from langchain_chroma import Chroma

# 데이터를 처음 저장할 때 
# database = Chroma.from_documents(
#     documents=document_list, 
#     embedding=embedding,
#     collection_name='chroma-tax',
#     persist_directory='./chroma'
# )

# 이미 저장된 데이터를 사용할 때 
database = Chroma(
    collection_name='chroma-tax',
    persist_directory='./chroma',
    embedding_function=embedding
)

In [48]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'

# `k` 값을 조절해서 얼마나 많은 데이터를 불러올지 결정
retrieved_docs = database.similarity_search(query, k=3)

In [49]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [50]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [51]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [55]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [59]:
ai_message = qa_chain({"query": query})

In [ ]:
ai_message = qa_chain.invoke(query)

In [60]:
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 직장인의 소득세는 약 532만원입니다. 이는 근로소득공제와 세액공제를 적용한 후의 금액입니다. 정확한 계산은 개인의 공제 항목에 따라 달라질 수 있습니다.'}